## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC

In [2]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [3]:
inputdata = a2.preprocess2(gmbfile)
gmbfile.close()
inputdata[20:40]

[['39', '2.0', 'slogan', 'NNS', 'O'],
 ['42', '2.0', 'bush', 'NNP', 'B-per'],
 ['43', '2.0', 'number', 'NN', 'O'],
 ['44', '2.0', 'one', 'CD', 'O'],
 ['45', '2.0', 'terrorist', 'NN', 'O'],
 ['49', '2.0', 'stop', 'VB', 'O'],
 ['51', '2.0', 'bombing', 'NNS', 'O'],
 ['55', '3.0', 'march', 'VBD', 'O'],
 ['58', '3.0', 'house', 'NNS', 'O'],
 ['60', '3.0', 'parliament', 'NN', 'O'],
 ['63', '3.0', 'rally', 'NN', 'O'],
 ['65', '3.0', 'hyde', 'NNP', 'B-geo'],
 ['66', '3.0', 'park', 'NNP', 'I-geo'],
 ['68', '4.0', 'police', 'NNS', 'O'],
 ['69', '4.0', 'put', 'VBD', 'O'],
 ['71', '4.0', 'number', 'NN', 'O'],
 ['73', '4.0', 'marcher', 'NNS', 'O'],
 ['75', '4.0', '10,000', 'CD', 'O'],
 ['77', '4.0', 'organizer', 'NNS', 'O'],
 ['78', '4.0', 'claim', 'VBD', 'O']]

## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [4]:
instances = a2.create_instances(inputdata)

In [5]:
instances[20:40]

[Class: gpe Features: ['<s5>', '<s4>', '<s3>', '<s2>', '<s1>', 'week', 'restart', 'part', 'conversion', 'process'],
 Class: geo Features: ['process', 'conversion', 'part', 'restart', 'week', 'nuclear', 'plant', '<e5>', '<e4>', '<e3>'],
 Class: gpe Features: ['<s5>', '<s4>', '<s3>', '<s2>', '<s1>', 'official', 'say', 'expect', 'get', 'access'],
 Class: tim Features: ['plant', 'part', 'sensitive', 'sealed', 'access', 'surveillance', 'system', 'begin', 'function', '<e5>'],
 Class: org Features: ['plant', 'part', 'sensitive', 'sealed', 'access', 'surveillance', 'system', 'begin', 'function', '<e5>'],
 Class: org Features: ['<s5>', '<s4>', '<s3>', '<s2>', '<s1>', 'backing', 'threaten', 'refer', 'could', 'impose'],
 Class: gpe Features: ['<s5>', '<s4>', '<s3>', '<s2>', '<s1>', 'backing', 'threaten', 'refer', 'could', 'impose'],
 Class: gpe Features: ['refer', 'threaten', 'backing', '<s5>', '<s4>', 'could', 'impose', 'sanction', 'find', 'violate'],
 Class: org Features: ['refer', 'threaten', 

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [6]:
bigdf = a2.create_table(instances)
bigdf[20:40]

,class,0,1,2,3,4,5,6,7,8,...,3786,3787,3788,3789,3790,3791,3792,3793,3794,3795
20,gpe,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,geo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,gpe,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,tim,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,org,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,org,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,gpe,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,gpe,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,org,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,gpe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(      0     1     2     3     4     5     6     7     8     9     ...  3786  \
 0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 1      0.0   0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 2      0.0   0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 4      0.0   0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 ...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
 5474   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 5475   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  ...   0.0   
 5476   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 5477   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 5478   0.0   0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 
       3787  3788  3789  3790  3791  3

In [8]:
len(test_y) / (len(test_y) + len(train_y))

0.2000292013432618

In [9]:
len(test_X) / (len(test_X) + len(train_X))

0.2000292013432618

In [10]:
test_y[0]

'geo'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [11]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

In [12]:
train_predictions

array(['tim', 'per', 'per', ..., 'tim', 'tim', 'geo'], dtype=object)

In [13]:
train_y

0       tim
1       per
2       per
3       gpe
4       gpe
       ... 
5474    tim
5475    gpe
5476    tim
5477    tim
5478    geo
Name: class, Length: 5479, dtype: object

In [14]:
test_predictions

array(['org', 'tim', 'tim', ..., 'per', 'per', 'per'], dtype=object)

In [15]:
test_y

0       geo
1       per
2       per
3       per
4       eve
       ... 
1365    tim
1366    geo
1367    org
1368    org
1369    tim
Name: class, Length: 1370, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [16]:
a2.confusion_matrix(test_y, test_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,1,0,2,2,0,2,2,1
eve,0,1,4,3,0,0,0,2
geo,0,2,190,45,0,50,40,63
gpe,3,0,48,76,0,35,45,29
nat,0,0,1,0,0,1,0,1
org,0,1,82,37,0,54,41,36
per,0,1,60,49,0,41,62,27
tim,4,2,88,30,1,26,33,46


In [17]:
a2.confusion_matrix(train_y, train_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,37,0,1,1,0,1,1,1
eve,0,32,1,0,0,0,0,1
geo,1,2,1431,54,0,61,54,62
gpe,0,0,79,796,0,30,66,23
nat,0,0,1,0,16,0,0,0
org,2,0,86,31,0,756,45,46
per,1,0,67,54,0,47,670,28
tim,1,1,125,35,1,45,38,648


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')